#### Load useful libraries and data

In [1]:
from transformers import pipeline
import pandas as pd
import numpy as np
from tqdm import tqdm


In [2]:
data = pd.read_csv(
    "data/SMSSpamCollection.txt",
    encoding="utf-8",
    header=None,
    delimiter="\t",
    names=["target", "text"],
)

In [3]:
data.head(5)

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
print(f'There are {data.shape[0]} rows in the dataset')

There are 5572 rows in the dataset


### Preparing the pipeline in one-line of code!

In [10]:
classifier = pipeline("zero-shot-classification",device = 0)

No model was supplied, defaulted to FacebookAI/roberta-large-mnli and revision 130fb28 (https://huggingface.co/FacebookAI/roberta-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


### Making Predictions

This model works best with informative labels, spam/ham are not so informative. Using spam/ham leads to a Hamming loss of 53% vs using click bait/written by humans leading to 19%

Can you find better label descriptions?

In [ ]:
category_map = {"spam":"click bait", "ham":"written by humans"}

In [7]:
candidate_labels = list(category_map.values())
predictedCategories = []
trueCategories = []
for i in tqdm(range(100)):
    text = data.iloc[i,]['text']
    cat = [data.iloc[i,]['target']]
    res = classifier(text, candidate_labels, multi_label=False)
    labels = res['labels'] 
    scores = res['scores'] #extracting the scores associated with the labels
    res_dict = {label : score for label,score in zip(labels, scores)}
    sorted_dict = dict(sorted(res_dict.items(), key=lambda x:x[1],reverse = True)) #sorting the dictionary of labels in descending order based on their score
    categories  = next(k for i, (k,v) in enumerate(sorted_dict.items()))

    predictedCategories.append(categories)
    trueCats = [category_map[x] for x in cat]
    trueCategories.append(trueCats)

100%|██████████| 100/100 [00:38<00:00,  2.57it/s]


In [9]:
for y_true, y_pred in zip(trueCategories[:3], predictedCategories[:3]):
    print(f'True Categories {y_true}')
    print(f'Predicted Categories {y_pred}')
    print('#'*50)

True Categories ['written by humans']
Predicted Categories written by humans
##################################################
True Categories ['written by humans']
Predicted Categories written by humans
##################################################
True Categories ['click bait']
Predicted Categories written by humans
##################################################


In [11]:
y_pred

'written by humans'

In [12]:
predictedCategories

['written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'click bait',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by humans',
 'written by huma

### Hamming Loss
The Hamming loss is the fraction of labels that are incorrectly predicted.

In [13]:
from sklearn.metrics import hamming_loss
print(f'The hamming loss is {hamming_loss(trueCategories,predictedCategories):.4f} compared to 0.0237 from the last trained model in Notebook 1')

The hamming loss is 0.1900 compared to 0.0237 from the last trained model in Notebook 1


## Check your understanding

Can you find better "labels" for the category description improve the Hamming Loss?

In [16]:
category_map = {"spam":"advertising", "ham":"written by humans"}

candidate_labels = list(category_map.values())
predictedCategories = []
trueCategories = []
for i in tqdm(range(100)):
    text = data.iloc[i,]['text']
    cat = [data.iloc[i,]['target']]
    res = classifier(text, candidate_labels, multi_label=False)
    labels = res['labels'] 
    scores = res['scores'] #extracting the scores associated with the labels
    res_dict = {label : score for label,score in zip(labels, scores)}
    sorted_dict = dict(sorted(res_dict.items(), key=lambda x:x[1],reverse = True)) #sorting the dictionary of labels in descending order based on their score
    categories  = next(k for i, (k,v) in enumerate(sorted_dict.items()))

    predictedCategories.append(categories)
    trueCats = [category_map[x] for x in cat]
    trueCategories.append(trueCats)
    
for y_true, y_pred in zip(trueCategories[:3], predictedCategories[:3]):
    print(f'True Categories {y_true}')
    print(f'Predicted Categories {y_pred}')
    print('#'*50)

100%|██████████| 100/100 [00:39<00:00,  2.50it/s]

True Categories ['written by humans']
Predicted Categories advertising
##################################################
True Categories ['written by humans']
Predicted Categories written by humans
##################################################
True Categories ['advertising']
Predicted Categories written by humans
##################################################


In [17]:
from sklearn.metrics import hamming_loss
print(f'The hamming loss is {hamming_loss(trueCategories,predictedCategories):.4f} compared to 0.0237 from the last trained model in Notebook 1')

The hamming loss is 0.1600 compared to 0.0237 from the last trained model in Notebook 1
